In [1]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Deploy ADK Agent in AI Engine and Agentspace

This notebook provides a step-by-step to deploy the Data Assistant ADK agent on Vertex AI agent engine, and deploy to Agentspace

- Import Libraries and Setup env_vars
- Package and Deploy Agent on Agent Engine
- Deploy on Agentspace

### Setup and Config

In [2]:
# Basic Libraries
import os 
import vertexai

# AI Engine on Vertex AI 
from vertexai import agent_engines

# Library for AI Engine with ADK
from vertexai.preview import reasoning_engines

# To view JSON response formatted 
import json
from IPython.display import JSON


In [3]:
# Load environment variables from data_assistant .env file 

from dotenv import load_dotenv
load_dotenv('./data_assistant/.env')
env_vars = {}

In [4]:
# Load Variables on env_vars dict to be used when creating the Agent

env_vars["GOOGLE_GENAI_USE_VERTEXAI"] = os.getenv("GOOGLE_GENAI_USE_VERTEXAI")
env_vars["GOOGLE_CLOUD_PROJECT_ID"] = os.getenv("GOOGLE_CLOUD_PROJECT_ID")
env_vars["GOOGLE_CLOUD_LOCATION_AGENT"] = os.getenv("GOOGLE_CLOUD_LOCATION_AGENT")
env_vars["GOOGLE_CLOUD_STORAGE_BUCKET"] = os.getenv("GOOGLE_CLOUD_STORAGE_BUCKET")


env_vars["BQ_PROJECT_ID"] = os.getenv("BQ_PROJECT_ID")
env_vars["BQ_DATASET_ID"] = os.getenv("BQ_DATASET_ID")
env_vars["BQ_ROOT_MODEL"] = os.getenv("BQ_ROOT_MODEL")
env_vars["BQ_TOOL_MODEL"] = os.getenv("BQ_TOOL_MODEL")
env_vars["BQ_LOCATION"] = os.getenv("BQ_LOCATION")
env_vars["BQ_METADATA_MODE"] = os.getenv("BQ_METADATA_MODE")
env_vars

{'GOOGLE_GENAI_USE_VERTEXAI': 'True',
 'GOOGLE_CLOUD_PROJECT_ID': 'speca-sandbox',
 'GOOGLE_CLOUD_LOCATION_AGENT': 'us-central1',
 'GOOGLE_CLOUD_STORAGE_BUCKET': 'gs://speca-data-assistant-staging',
 'BQ_PROJECT_ID': 'speca-sandbox',
 'BQ_DATASET_ID': 'legaldata',
 'BQ_ROOT_MODEL': 'gemini-2.5-flash-preview-04-17',
 'BQ_TOOL_MODEL': 'gemini-2.0-flash-001',
 'BQ_LOCATION': 'us',
 'BQ_METADATA_MODE': 'OFF'}

In [6]:
# Instantiate Vertex AI

vertexai.init(
    project=env_vars["GOOGLE_CLOUD_PROJECT_ID"],
    location=env_vars["GOOGLE_CLOUD_LOCATION_AGENT"],
    staging_bucket=env_vars["GOOGLE_CLOUD_STORAGE_BUCKET"],
)

### Agent ADK Package and Deploy

In [7]:
# Importong root_agent from local data_assistant folder
from data_assistant.agent import root_agent

In [43]:
# Requirements file
requirements = [
    "google-cloud-aiplatform[agent_engines]",
    "google-adk==0.5.0",
    "cloudpickle",
    "pydantic",
    "google-cloud-bigquery",
    "pandas",
    "db-dtypes"
]

In [44]:
# Extra packages from foder data_assistant
extra_packages = ["data_assistant"]

In [45]:
# Agent Metadata 

agent_display_name = "Legal Data Agent"

agent_description = """Você é um assistente especializado em consultar dados de processos judiciais uma base de dados de processos."""

In [46]:
# Instantiate the Assistant as an ADK App 
app = reasoning_engines.AdkApp(
    agent=root_agent,
)

In [47]:
# For testing purposes create a session
session = app.create_session(user_id="user_1")
session

Session(id='06c35a27-880f-46ba-b36c-856bc0c5c019', app_name='default-app-name', user_id='user_1', state={}, events=[], last_update_time=1747870265.646553)

In [48]:
# Run a simple query
for event in app.stream_query(
    user_id="user_1",
    session_id=session.id,
    message="Quantos processos em 2023 por assunto?",
):
    print(event)

{'invocation_id': 'e-3328baa4-9126-4ef4-b91a-339636326e18', 'author': 'bq_data_assistant', 'actions': {'state_delta': {'database_settings': {'bq_project_id': 'speca-sandbox', 'bq_dataset_id': 'legaldata', 'bq_ddl_schema': "CREATE OR REPLACE TABLE `speca-sandbox.legaldata.decisoes` (\n  `nro_processo` STRING,\n  `classe` STRING,\n  `assunto` STRING,\n  `magistrado` STRING,\n  `comarca` STRING,\n  `foro` STRING,\n  `vara` STRING,\n  `data_disp` STRING,\n  `texto_decisao` STRING,\n  `instancia` STRING,\n  `resultado` STRING\n);\n\nCREATE OR REPLACE TABLE `speca-sandbox.legaldata.processos` (\n  `cod_empresa` STRING,\n  `nro_processo` STRING,\n  `link_processo` STRING,\n  `nome_parte` STRING,\n  `tipo_participacao` STRING,\n  `classe_processo` STRING,\n  `assunto_processo` STRING,\n  `data_distribuicao` DATETIME,\n  `local_distribuicao` STRING,\n  `decisao_primeira` STRING,\n  `valor_contingencia` FLOAT,\n  `prognostico_risco` STRING\n);\n\n-- Example values for table `speca-sandbox.legald

{'content': {'parts': [{'function_call': {'id': 'adk-71faf284-d63a-4002-a39e-6041d9151994', 'args': {'question': 'Quantos processos em 2023 por assunto?'}, 'name': 'bq_nl2sql'}}], 'role': 'model'}, 'invocation_id': 'e-3328baa4-9126-4ef4-b91a-339636326e18', 'author': 'bq_data_assistant', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': set(), 'id': 'cIrQzr1c', 'timestamp': 1747870268.70846}

 sql: SELECT
    COUNT(p.nro_processo) AS quantidade_processos,
    p.assunto_processo
  FROM
    `speca-sandbox.legaldata.processos` AS p
  WHERE EXTRACT(YEAR FROM p.data_distribuicao) = 2023
  GROUP BY 
    p.assunto_processo
  LIMIT 80
{'content': {'parts': [{'function_response': {'id': 'adk-71faf284-d63a-4002-a39e-6041d9151994', 'name': 'bq_nl2sql', 'response': {'result': 'SELECT\n    COUNT(p.nro_processo) AS quantidade_processos,\n    p.assunto_processo\n  FROM\n    `speca-sandbox.legaldata.processos` AS p\n  WHERE EXTRACT(YEAR FROM p.

{'content': {'parts': [{'function_call': {'id': 'adk-5b1e0f20-9bbc-4e58-8c5f-25f7af864e97', 'args': {'sql_string': 'SELECT\n    COUNT(p.nro_processo) AS quantidade_processos,\n    p.assunto_processo\n  FROM\n    `speca-sandbox.legaldata.processos` AS p\n  WHERE EXTRACT(YEAR FROM p.data_distribuicao) = 2023\n  GROUP BY \n    p.assunto_processo\n  LIMIT 80'}, 'name': 'run_bigquery_validation'}}], 'role': 'model'}, 'invocation_id': 'e-3328baa4-9126-4ef4-b91a-339636326e18', 'author': 'bq_data_assistant', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': set(), 'id': 'ZHMyeegF', 'timestamp': 1747870274.845221}

 run_bigquery_validation final_result: 
 {'query_result': [{'quantidade_processos': 18, 'assunto_processo': 'DIREITO PROCESSUAL CIVIL E DO TRABALHO'}, {'quantidade_processos': 5, 'assunto_processo': 'Liminar'}, {'quantidade_processos': 1, 'assunto_processo': 'Atos Administrativos'}, {'quantidade_processos': 25, 'assunto_proce

In [49]:
json_answer = event['content']['parts'][0]['text']

# in case that the output doesnt bring a JSON
try:  
    json_answer = json_answer.replace('```json','').replace('```','').replace('\n','').replace('\\n','')
    JSON(json.loads(json_answer))
except :
    print(json_answer)

Em 2023, a quantidade de processos por assunto é a seguinte:*   DIREITO PROCESSUAL CIVIL E DO TRABALHO: 18*   Liminar: 5*   Atos Administrativos: 1*   Práticas Abusivas: 25*   Responsabilidade do Fornecedor: 17*   Interpretação / Revisão de Contrato: 1*   Indenização por Dano Material: 27*   Indenização por Dano Moral: 29*   Provas em geral: 3*   Proteção de dados pessoais (LGPD): 11*   Perdas e Danos: 5*   Defeito, nulidade ou anulação: 9*   Concorrência desleal: 4*   Marca: 6*   DIREITO CIVIL: 5*   Direito de Imagem: 5*   Obrigações: 8*   Bancários: 1*   Contratos Bancários: 1*   Oferta e Publicidade: 1*   Dever de Informação: 5*   Crédito Direto ao Consumidor - CDC: 1*   Concurso de Credores: 1*   Petição intermediária: 4*   Estaduais: 2*   Enriquecimento sem Causa: 1*   Prestação de Serviços: 1*   Inventário e Partilha: 1*   Irregularidade no atendimento: 1*   Contrafação: 1*   Compra e Venda: 2*   Rescisão do contrato e devolução do dinheiro: 1*   Direito Autoral: 1*   Revisão de 

In [50]:
## (OPTIONAL) - Retrieve all existent Agent Engine resource.names (Agents)
for agent in agent_engines.list():
    print(f"Agent: {agent.display_name} [{agent.resource_name}] created/updated at: {agent.update_time}" )

Agent: Legal Data Agent [projects/41865284455/locations/us-central1/reasoningEngines/299533355685249024] created/updated at: 2025-05-21 20:43:56.967645+00:00
Agent: Legal Data Assistant [projects/41865284455/locations/us-central1/reasoningEngines/1622465746225332224] created/updated at: 2025-05-21 18:01:54.856531+00:00


In [ ]:
## (OPTIONAL) - Update the agent at the same Agent Resource 
# agent_engines.update(
#     resource_name=remote_agent.resource_name,    # Required.
#     agent_engine = app,
#     extra_packages=extra_packages,      # Extra packages
#     display_name=agent_display_name,    # Display name  
#     description=agent_description,      # Description
#     env_vars=env_vars 
# )

In [51]:
# Deploy the Agent on AI Engine (This takes a few minutes)

remote_agent = agent_engines.create(
    app,                                # The Agente instantiated as ADK agent
    requirements=requirements,          # Requirements file
    extra_packages=extra_packages,      # Extra packages
    display_name=agent_display_name,    # Display name  
    description=agent_description,      # Description
    env_vars=env_vars                   # Env Vars dict
)

Identified the following requirements: {'google-cloud-aiplatform': '1.93.0', 'cloudpickle': '3.1.1', 'pydantic': '2.11.4'}


INFO:vertexai.agent_engines:Identified the following requirements: {'google-cloud-aiplatform': '1.93.0', 'cloudpickle': '3.1.1', 'pydantic': '2.11.4'}


The final list of requirements: ['google-cloud-aiplatform[agent_engines]', 'google-adk==0.5.0', 'cloudpickle', 'pydantic', 'google-cloud-bigquery', 'pandas', 'db-dtypes']


INFO:vertexai.agent_engines:The final list of requirements: ['google-cloud-aiplatform[agent_engines]', 'google-adk==0.5.0', 'cloudpickle', 'pydantic', 'google-cloud-bigquery', 'pandas', 'db-dtypes']


Using bucket speca-data-assistant-staging


...

INFO:vertexai.agent_engines:AgentEngine created. Resource name: projects/41865284455/locations/us-central1/reasoningEngines/6064140878719483904


To use this AgentEngine in another session:


INFO:vertexai.agent_engines:To use this AgentEngine in another session:


agent_engine = vertexai.agent_engines.get('projects/41865284455/locations/us-central1/reasoningEngines/6064140878719483904')


INFO:vertexai.agent_engines:agent_engine = vertexai.agent_engines.get('projects/41865284455/locations/us-central1/reasoningEngines/6064140878719483904')


In [52]:
# Retrieve ID of Agent Engine
remote_agent.resource_name

'projects/41865284455/locations/us-central1/reasoningEngines/6064140878719483904'

In [53]:
# Testing Remote Agent
session_remote = remote_agent.create_session(user_id="user_1")


In [54]:
session_remote["id"]

'5541587532258476032'

In [55]:
# Query on remote agent for test 
for event in remote_agent.stream_query(
    user_id="user_1",
    session_id=session_remote['id'],  
    message="Quantos processos em 2023 por assunto?",
):
   print(event)

{'invocation_id': 'e-4fe30c46-c7f4-4f0c-9e0b-118e1511ff78', 'author': 'bq_data_assistant', 'actions': {'state_delta': {'database_settings': {'bq_project_id': 'speca-sandbox', 'bq_dataset_id': 'legaldata', 'bq_ddl_schema': "CREATE OR REPLACE TABLE `speca-sandbox.legaldata.decisoes` (\n  `nro_processo` STRING,\n  `classe` STRING,\n  `assunto` STRING,\n  `magistrado` STRING,\n  `comarca` STRING,\n  `foro` STRING,\n  `vara` STRING,\n  `data_disp` STRING,\n  `texto_decisao` STRING,\n  `instancia` STRING,\n  `resultado` STRING\n);\n\nCREATE OR REPLACE TABLE `speca-sandbox.legaldata.processos` (\n  `cod_empresa` STRING,\n  `nro_processo` STRING,\n  `link_processo` STRING,\n  `nome_parte` STRING,\n  `tipo_participacao` STRING,\n  `classe_processo` STRING,\n  `assunto_processo` STRING,\n  `data_distribuicao` DATETIME,\n  `local_distribuicao` STRING,\n  `decisao_primeira` STRING,\n  `valor_contingencia` FLOAT,\n  `prognostico_risco` STRING\n);\n\n-- Example values for table `speca-sandbox.legald

In [56]:
# To view JSON response formatted 
json_answer = event['content']['parts'][0]['text']

# in case that the output doesnt bring a JSON
try:  
    json_answer = json_answer.replace('```json','').replace('```','').replace('\n','').replace('\\n','') 
    JSON(json.loads(json_answer))
except :
    print(json_answer)

Em 2023, a quantidade de processos por assunto é a seguinte:*   DIREITO PROCESSUAL CIVIL E DO TRABALHO: 18*   Liminar: 5*   Atos Administrativos: 1*   Práticas Abusivas: 25*   Responsabilidade do Fornecedor: 17*   Interpretação / Revisão de Contrato: 1*   Indenização por Dano Material: 27*   Indenização por Dano Moral: 29*   Provas em geral: 3*   Proteção de dados pessoais (LGPD): 11*   Perdas e Danos: 5*   Defeito, nulidade ou anulação: 9*   Concorrência desleal: 4*   Marca: 6*   DIREITO CIVIL: 5*   Direito de Imagem: 5*   Obrigações: 8*   Bancários: 1*   Contratos Bancários: 1*   Oferta e Publicidade: 1*   Dever de Informação: 5*   Crédito Direto ao Consumidor - CDC: 1*   Concurso de Credores: 1*   Petição intermediária: 4*   Estaduais: 2*   Enriquecimento sem Causa: 1*   Prestação de Serviços: 1*   Inventário e Partilha: 1*   Irregularidade no atendimento: 1*   Contrafação: 1*   Compra e Venda: 2*   Rescisão do contrato e devolução do dinheiro: 1*   Direito Autoral: 1*   Revisão de 

### Deploy Agent on Agentspace

This is a temporary step, due the official way to do this still working in progress

In [57]:
import subprocess
import json
import requests

# The ID of your Google Cloud project.
project_id = env_vars["GOOGLE_CLOUD_PROJECT_ID"] 

# The ID of the Agentspace app (see: https://cloud.google.com/agentspace/agentspace-enterprise/docs/assistant ).
app_id = "legal-assistant-poc_1743187751049"

# The display name of the agent.
display_name = "Legal Data Assistant"

# The description of the agent, displayed on the frontend; it is only for the user’s benefit.
description = "Agente especializado em buscar dados estruturados na base de processos"

# The description of the agent used by the LLM to route requests to the agent.
# Must properly describe what the agent does. Never shown to the user.
tool_description = "Agent to search legal data on database, used for quantitative questions like, how many, which ones, sum of for legal lawsuits"

# The ID of the reasoning engine endpoint where the ADK agent is deployed (Resource.name).
adk_deployment_name = "projects/41865284455/locations/us-central1/reasoningEngines/1622465746225332224" # The remote_agent.resource_name Deployed
 

In [58]:
# Get the access token from gcloud
try:
    access_token = subprocess.check_output(
        "gcloud auth print-access-token", shell=True, text=True
    ).strip()
except subprocess.CalledProcessError as e:
    print(f"Error getting access token: {e}")
    exit()

# API endpoint
url = f"https://discoveryengine.googleapis.com/v1alpha/projects/{project_id}/locations/global/collections/default_collection/engines/{app_id}/assistants/default_assistant/agents"

# Headers
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json",
    "X-Goog-User-Project": project_id,
}

# Data payload
data = {
    "displayName": display_name,
    "description": description,
    "adk_agent_definition": {
        "tool_settings": {"tool_description": tool_description},
        "provisioned_reasoning_engine": {"reasoning_engine": adk_deployment_name},
    },
}

In [59]:
# Make the POST request
response = requests.post(url, headers=headers, data=json.dumps(data))

# Print the response
print(f"Status Code: {response.status_code}")
print(f"Response JSON: \n")
JSON(response.json())

Status Code: 200
Response JSON: 



<IPython.core.display.JSON object>